In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install mnist scikit-learn opencv-python tensorflow

In [2]:
import numpy as np
import mnist
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf

In [14]:
# import tensorflow as tf

# # Load the MNIST dataset
# mnist = tf.keras.datasets.mnist
# (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [15]:
# train_images = train_images / 255.0
# test_images = test_images / 255.0

# Reshape the images to add a color channel dimension
# train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
# test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

In [ ]:
# Load the MNIST dataset
# train_images = mnist.train_images()
# train_labels = mnist.train_labels()
# test_images = mnist.test_images()
# test_labels = mnist.test_labels()

# Flatten and normalize the images
# train_images = train_images.reshape(-1, 28*28) / 255.0
# test_images = test_images.reshape(-1, 28*28) / 255.0
# print(train_images[0])
# One-hot encode the labels
# num_classes = 10
# train_labels = np.eye(num_classes)[train_labels]
# test_labels = np.eye(num_classes)[test_labels]

In [8]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(
#     rotation_range=10,        # Randomly rotate images in the range (degrees, 0 to 180)
#     width_shift_range=0.1,    # Randomly shift images horizontally (fraction of total width)
#     height_shift_range=0.1,   # Randomly shift images vertically (fraction of total height)
#     zoom_range=0.1,           # Randomly zoom images
#     shear_range=0.1           # Randomly shear images
# )

# # Fit the data generator on the training images
# datagen.fit(train_images)

In [16]:
# train_labels = train_labels.astype(int)
# test_labels = test_labels.astype(int)

In [18]:
import numpy as np
import tensorflow as tf

# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the images to the range [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Reshape the images to add a color channel dimension
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

# Ensure labels are integers
train_labels = train_labels.astype(int)
test_labels = test_labels.astype(int)

# One-hot encode labels
def one_hot_encode(labels, num_classes):
    return np.eye(num_classes)[labels]

train_labels_one_hot = one_hot_encode(train_labels, 10)
test_labels_one_hot = one_hot_encode(test_labels, 10)

# Data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)

datagen.fit(train_images)

# Define a simple feedforward neural network
input_size = 28 * 28
hidden_size = 128
output_size = 10
learning_rate = 0.1

# Initialize weights and biases
np.random.seed(0)
weights_input_hidden = np.random.randn(input_size, hidden_size)
biases_hidden = np.zeros(hidden_size)
weights_hidden_output = np.random.randn(hidden_size, output_size)
biases_output = np.zeros(output_size)

# Softmax function
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))  # Subtract the max for numerical stability
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

# Training loop
num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    total_loss = 0
    num_batches = len(train_images) // batch_size

    for X_batch, y_batch in datagen.flow(train_images, train_labels_one_hot, batch_size=batch_size):
        for i in range(len(X_batch)):
            # Forward pass
            input_layer = X_batch[i].flatten()  # Flatten the input image
            hidden_layer_input = np.dot(input_layer, weights_input_hidden) + biases_hidden
            hidden_layer_output = 1 / (1 + np.exp(-hidden_layer_input))  # Sigmoid activation
            output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + biases_output
            output_layer_output = softmax(output_layer_input)

            # Calculate loss (cross-entropy)
            loss = -np.sum(y_batch[i] * np.log(output_layer_output))
            total_loss += loss

            # Backpropagation
            output_error = output_layer_output - y_batch[i]
            hidden_error = np.dot(output_error, weights_hidden_output.T)
            hidden_delta = hidden_error * hidden_layer_output * (1 - hidden_layer_output)
            
            weights_hidden_output -= learning_rate * np.outer(hidden_layer_output, output_error)
            biases_output -= learning_rate * output_error
            weights_input_hidden -= learning_rate * np.outer(input_layer, hidden_delta)
            biases_hidden -= learning_rate * hidden_delta
        
        if len(X_batch) < batch_size:
            break  # To avoid processing incomplete batch at the end

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / num_batches}")

# Testing
correct = 0
for i in range(len(test_images)):
    input_layer = test_images[i].flatten()  # Flatten the input image
    hidden_layer_input = np.dot(input_layer, weights_input_hidden) + biases_hidden
    hidden_layer_output = 1 / (1 + np.exp(-hidden_layer_input))  # Sigmoid activation
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + biases_output
    output_layer_output = softmax(output_layer_input)
    prediction = np.argmax(output_layer_output)
    if prediction == np.argmax(test_labels_one_hot[i]):
        correct += 1

accuracy = correct / len(test_images) * 100
print(f"Test Accuracy: {accuracy:.2f}%")


KeyboardInterrupt: 

In [ ]:
image = cv2.imread('/content/drive/MyDrive/hand digit digit recog/input image/six.PNG', cv2.IMREAD_GRAYSCALE)
resized_image = cv2.resize(image, (28, 28))
normalized_image = resized_image / 255.0

# input_image = image.reshape(-1, 28*28) / 255.0
flattened_image = normalized_image.reshape(1, -1)
flattened_image[flattened_image >= 1] = 0

In [ ]:
input_layer = flattened_image
hidden_layer_input = np.dot(input_layer, weights_input_hidden) + biases_hidden
hidden_layer_output = 1 / (1 + np.exp(-hidden_layer_input))  # Sigmoid activation
output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + biases_output
output_layer_output = softmax(output_layer_input)
prediction = np.argmax(output_layer_output)

print(prediction)

5
